In [8]:
import pandas as pd
import re
import numpy as np

In [2]:
inp_file = './results_pipeline_tr_filtered/RDRS_train_filtered_4_NER_2391028.out'

In [3]:
#если несколько фолдов
from os.path import dirname, realpath, splitext, basename

folds_sep = '(?<=\n)\d+\n'
with open(inp_file, 'r') as f:
    txt = f.read()
    
seps = '\n\d\n|^\d+\n'
fold_nums = re.findall(seps, txt)
fold_nums = list(map(lambda x: re.sub('\D', '', x), fold_nums))


txt_parts = re.split(folds_sep, txt)
txt_parts = [txt_part for txt_part in txt_parts if txt_part]
if len(txt_parts)!=len(fold_nums):
    if txt_parts[0].find('Evaluation')<0:
        txt_parts = txt_parts[1:]
    elif txt_parts[-1].find('Evaluation')<0:
        txt_parts = txt_parts[:-1]
    else:
        raise AssertionError
    assert len(txt_parts)==len(fold_nums)

filename = splitext(basename(inp_file))[0]
for i, fold in enumerate(fold_nums):
    fold_filename = fold + '_' + filename + '.out'
    with open(dirname(realpath(inp_file)) + '/' + fold_filename, 'w') as f:
        f.write(txt_parts[i])

In [4]:
#если несколько фолдов
import os

for file in os.listdir(dirname(realpath(inp_file))):
    if not file[0].isdigit() or file.find('csv')>0:
        continue
    fold_num = file.split('_')[0]
    #читаем файл, делим на сперт репорты
    with open(dirname(realpath(inp_file)) + '/' + file, 'r') as f:
        txt = f.read()
    txt_parts = re.split('Evaluation\n\n|--------------------------------------------------', txt)
    txt_parts = [txt_part for txt_part in txt_parts if txt_part]
    spert_reports = [txt_part for txt_part in txt_parts if re.search('--- Entities .+\s+---', txt_part)]

    spert_reports = [re.sub('\n\s+', '\n', rep) for rep in spert_reports]


    #основной код
    def count_symbols_before_first_whitespace(matchobj):
        num_symbols = matchobj.span()[0]
        symbols_before_first_whitespace.append(num_symbols)
        return '<'+str(num_symbols)+'>'

    def make_valid_margin(matchobj):
        margin = max_margin - int(matchobj.group(0)[1:-1])
        return ' '*margin

    def delete_n_first_whitespaces(matchobj):
        columns = matchobj.group()
        first_n_whitespaces = re.match('\s+', columns).span()[1]
        return ' '*(first_n_whitespaces-delete_n_first_whitespaces.n)+columns[first_n_whitespaces:]



    for rep_id in range(len(spert_reports)):
        sents = re.split('\n+', spert_reports[rep_id])
        #sents = spert_reports[rep_id].split('\n')
        symbols_before_first_whitespace = []
        for sent_id in range(1, len(sents)):
            #считаем кол-во символов до первого пробела в каждом предложении, 
            #заменяем первую череду пробелов в предложении
            #на это кол-во символов
            if sents[sent_id].find('---')>=0 or re.search('\d\.\d', sents[sent_id]) is None and sents[sent_id].find('f1-score')<0:
                sents[sent_id] = '\n' + sents[sent_id]
                continue
            if sents[sent_id].find('MainThread')>=0:
                break
            sents[sent_id] = re.sub('\s+', count_symbols_before_first_whitespace, sents[sent_id], count=1)
            sents[sent_id] = re.sub('(?<=\d)\s+', ' '*11, sents[sent_id], count=1)
            #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
            sents[sent_id] = re.sub('(?<=\W0\.00)\s+', ' '*12, sents[sent_id], count=1)
            delete_n_first_whitespaces.n=2
            sents[sent_id] = re.sub('\s+\d+\.\d+\s+\d+$', delete_n_first_whitespaces, sents[sent_id])
            sents[sent_id] = re.sub('\s+(\d+)$', ' '*9+r'\1', sents[sent_id])
            #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
            sents[sent_id] = re.sub('(?<=\W0\.00)\s+(\d+)$', ' '*10+r'\1', sents[sent_id])

        max_margin = max(symbols_before_first_whitespace)+4
        spert_reports[rep_id] = '\n'.join(sents)
        spert_reports[rep_id] = re.sub('<\d+>', make_valid_margin, spert_reports[rep_id])

    #дальше смотрим только NER результат
    NER_test_res = re.findall('{[^{}]+}', txt)[-1]
    #дампим вывод
    out_txt = "====NER result====\n\n"
    out_txt +=  NER_test_res
    out_txt += '\n\n====RE results====\n\n'
    ##   Если всего 2 сперт репорта, то нужно оставить эти строчки
    out_txt = out_txt + '====Gold NER====\n' + spert_reports[0] + '====Pred NER====\n' + spert_reports[1]
    out_file = dirname(realpath(inp_file)) + '/' + 'pretty_' + file
    ##
    
    with open(out_file, 'w') as f:
        f.write(out_txt)

In [5]:
#если несколько фолдов
#сделаем таблички, а также агрегируем точности
import pandas as pd

NER_res_dfs = []
RE_gold_NER_dfs = []
RE_pred_NER_dfs = []

for file in os.listdir(dirname(realpath(inp_file))):
    if file.find('pretty_')<0:
        continue
    with open(dirname(realpath(inp_file)) + '/' + file, 'r') as f:
        txt = f.read()
    file = file.replace('pretty_', '')
    fold_num = file.split('_')[0]
    file = re.sub('\d+_', '', file)
    all_eval_parts = re.split('====[^=]+====', txt)
    NER_res, RE_gold_NER, RE_pred_NER = [part for part in all_eval_parts if re.search('[^\n]', part)]
    #NER
    NER_res = re.sub("\n|\s|{|}|'", '', NER_res)
    ents_d = {ent_res.split(':')[0]:float(ent_res.split(':')[1]) for ent_res in NER_res.split(',')}
    NER_df = pd.DataFrame({'f1_conll': ents_d.values()}, index=ents_d.keys())
    NER_res_dfs.append(NER_df)
    out_file = realpath(dirname(realpath(inp_file))) + '/' + fold_num + '_' + 'NER' + '_' + file.replace('.out', '.csv')
    NER_df.to_csv(out_file)
    #RE gold NER
    RE_gold_NER = RE_gold_NER.split('A relation is considered correct if the relation type and the two related entities are predicted correctly (in span and entity type)')[-1]
    RE_gold_NER = re.sub('Parse.+\n', '', RE_gold_NER)
    RE_gold_NER = [row for row in RE_gold_NER.split('\n') if row]
    col_names = re.split('\s+', RE_gold_NER[0])[1:]
    rel_df =  pd.DataFrame(columns=col_names)
    for row in RE_gold_NER[1:]:
        row = re.split('\s+', row)
        rel_df = rel_df.append(pd.Series(map(float, row[1:]), index=col_names, name=row[0].replace('none_', '')))
    rel_df['support'] = rel_df['support'].apply(int)
    RE_gold_NER_dfs.append(rel_df)
    out_file = realpath(dirname(realpath(inp_file))) + '/' + fold_num + '_' + 'RE_gold_NER' + '_' + file.replace('.out', '.csv')
    rel_df.to_csv(out_file)
    #RE pred NER
    RE_pred_NER = RE_pred_NER.split('A relation is considered correct if the relation type and the two related entities are predicted correctly (in span and entity type)')[-1]
    RE_pred_NER = re.sub('Parse.+\n', '', RE_pred_NER)
    RE_pred_NER = [row for row in RE_pred_NER.split('\n') if row]
    col_names = re.split('\s+', RE_pred_NER[0])[1:]
    rel_df =  pd.DataFrame(columns=col_names)
    for row in RE_pred_NER[1:]:
        row = re.split('\s+', row)
        rel_df = rel_df.append(pd.Series(map(float, row[1:]), index=col_names, name=row[0].replace('none_', '')))
    rel_df['support'] = rel_df['support'].apply(int)
    RE_pred_NER_dfs.append(rel_df)
    out_file = realpath(dirname(realpath(inp_file))) + '/' + fold_num + '_' + 'RE_pred_NER' + '_' + file.replace('.out', '.csv')
    rel_df.to_csv(out_file)

In [6]:
#создаем одну общую таблицу

NER table

In [9]:
avg_NER_res_dfs = sum(NER_res_dfs)/len(NER_res_dfs)
avg_NER_res_dfs['std']=float(0)
for rel_type in avg_NER_res_dfs.index:
    #print(np.std(list(map(lambda x: x['f1_conll'][rel_type], NER_res_dfs))))
    avg_NER_res_dfs['f1_conll'][rel_type] = np.round(avg_NER_res_dfs['f1_conll'][rel_type], 2)
    avg_NER_res_dfs['std'][rel_type] = np.round(np.std(list(map(lambda x: x['f1_conll'][rel_type], NER_res_dfs))), 2)

In [60]:
avg_NER_res_dfs

,f1_conll,std
ADR,56.72,2.96
Disease_DisTypeDiseasename,87.23,2.72
Disease_DisTypeIndication,63.74,3.81
Medication_MedTypeDrugname,96.43,0.85
Medication_MedTypeSourceInfodrug,63.01,5.39


In [61]:
avg_NER_res_dfs.to_csv(realpath(dirname(realpath(inp_file)))+'/'+'avg_NER_res.csv')

RE tables

In [10]:
RE_f1_gold_NER_dfs = list(map(lambda x: pd.DataFrame(x['f1-score']),RE_gold_NER_dfs))
RE_f1_pred_NER_dfs = list(map(lambda x: pd.DataFrame(x['f1-score']),RE_pred_NER_dfs))

In [ ]:
import numpy as np

avg_RE_gold_NER_df = sum(RE_f1_gold_NER_dfs)/len(RE_f1_gold_NER_dfs)
avg_RE_gold_NER_df['std']=float(0)
for rel_type in avg_RE_gold_NER_df.index:
    print(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_gold_NER_dfs))))
    avg_RE_gold_NER_df['std'][rel_type] = np.round(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_gold_NER_dfs))), 3)

#avg_RE_pred_NER_df = sum(RE_f1_pred_NER_dfs)/len(RE_f1_pred_NER_dfs)

In [ ]:
avg_RE_gold_NER_df

In [238]:
s = '''import numpy as np

avg_RE_gold_NER_df = sum(RE_f1_gold_NER_dfs)/len(RE_f1_gold_NER_dfs)
avg_RE_gold_NER_df['std']=0
for rel_type in avg_RE_gold_NER_df.index:
    print(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_gold_NER_dfs))))
    avg_RE_gold_NER_df['std'][rel_type] = float(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_gold_NER_dfs)))'''

In [11]:
import numpy as np

avg_RE_pred_NER_df = sum(RE_f1_pred_NER_dfs)/len(RE_f1_pred_NER_dfs)
avg_RE_pred_NER_df['f1-score'] = np.round(avg_RE_pred_NER_df['f1-score'], 2)
avg_RE_pred_NER_df['std']=float(0)
for rel_type in avg_RE_pred_NER_df.index:
    avg_RE_pred_NER_df.at[rel_type, 'std'] = np.round(np.std(list(map(lambda x: x['f1-score'][rel_type], RE_pred_NER_dfs))), 2)

In [12]:
RE_f1_pred_NER_dfs

[]

In [12]:
avg_RE_pred_NER_df

,f1-score,std
ADR_Drugname_0,48.07,3.53
ADR_Drugname_1,52.74,4.44
Diseasename_Indication_0,10.63,7.45
Diseasename_Indication_1,52.12,7.72
Drugname_Diseasename_0,61.86,3.52
Drugname_Diseasename_1,77.90,2.09
Drugname_SourceInfodrug_0,40.09,6.42
Drugname_SourceInfodrug_1,49.97,3.65
macro,49.17,2.09
micro,57.66,2.00


In [51]:
avg_RE_pred_NER_df.to_csv(realpath(dirname(realpath(inp_file)))+'/'+'avg_RE_pred_NER_res.csv')

<h3>После 4 NER</h3>

In [65]:
#flatten df
sorted_rel_types = ['ADR_Drugname', 'Drugname_Diseasename', 'Drugname_SourceInfodrug', 'Diseasename_Indication']
pos_neg_order = ['0', '1']

sorted_rel_types = [[rel_type + '_' + rel_class for rel_class in pos_neg_order] for rel_type in sorted_rel_types]
sorted_rel_types = sum(sorted_rel_types, [])
flatten_RE_df = pd.DataFrame({k: [float(0)] for k in sorted_rel_types})
for rel_type in sorted_rel_types:
    flatten_RE_df[rel_type][0] = np.round(avg_RE_pred_NER_df['f1-score'][rel_type], 2)
out_file = realpath(dirname(realpath(inp_file))) + '/avg_flattened_pred_NER_RE_df.csv'
flatten_RE_df.to_csv(out_file, index=False)
flatten_RE_df

,ADR_Drugname_0,ADR_Drugname_1,Drugname_Diseasename_0,Drugname_Diseasename_1,Drugname_SourceInfodrug_0,Drugname_SourceInfodrug_1,Diseasename_Indication_0,Diseasename_Indication_1
0,48.07,52.74,61.86,77.9,40.09,49.97,10.63,52.12


<h3>До 4ых NER</h3>

In [52]:
#flatten df
sorted_rel_types = ['ADR_Drugname', 'Drugname_Diseasename', 'Drugname_SourceInfodrug', 'Diseasename_Indication']
pos_neg_order = ['0', '1']

sorted_rel_types = [[rel_type + '_' + rel_class for rel_class in pos_neg_order] for rel_type in sorted_rel_types]
sorted_rel_types = sum(sorted_rel_types, [])
flatten_RE_df = pd.DataFrame({k: [float(0)] for k in sorted_rel_types})
for rel_type in sorted_rel_types:
    flatten_RE_df[rel_type][0] = np.round(avg_RE_pred_NER_df['f1-score'][rel_type], 2)
out_file = realpath(dirname(realpath(inp_file))) + '/avg_flattened_pred_NER_RE_df.csv'
flatten_RE_df.to_csv(out_file, index=False)
flatten_RE_df

,ADR_Drugname_0,ADR_Drugname_1,Drugname_Diseasename_0,Drugname_Diseasename_1,Drugname_SourceInfodrug_0,Drugname_SourceInfodrug_1,Diseasename_Indication_0,Diseasename_Indication_1
0,43.62,50.54,63.49,78.04,43.04,50.66,10.58,48.77


In [243]:
avg_RE_gold_NER_df

,f1-score,std
ADR_Drugname_0,83.520,4.997
ADR_Drugname_1,91.672,4.094
Diseasename_Indication_0,33.140,4.411
Diseasename_Indication_1,87.124,0.681
Drugname_Diseasename_0,76.070,1.467
Drugname_Diseasename_1,88.372,1.281
Drugname_SourceInfodrug_0,81.014,1.697
Drugname_SourceInfodrug_1,91.946,0.307
macro,79.106,1.463
micro,85.080,0.986


In [181]:
pd.DataFrame(RE_gold_NER_dfs[0]['f1-score'])

,f1-score
Drugname_Diseasename_0,80.20
Drugname_SourceInfodrug_0,81.48
Diseasename_Indication_1,87.89
ADR_Drugname_1,94.71
Diseasename_Indication_0,37.44
Drugname_Diseasename_1,90.62
ADR_Drugname_0,89.17
Drugname_SourceInfodrug_1,92.12
micro,87.26
macro,81.70


In [179]:
sum(RE_gold_NER_dfs)/5

,precision,recall,f1-score,support
ADR_Drugname_0,88.448,79.772,83.810,183.4
ADR_Drugname_1,90.388,94.710,92.480,382.6
Diseasename_Indication_0,58.738,26.850,36.020,140.2
Diseasename_Indication_1,82.396,94.184,87.822,517.6
Drugname_Diseasename_0,82.660,75.410,78.522,430.6
Drugname_Diseasename_1,88.254,91.828,89.942,855.4
Drugname_SourceInfodrug_0,84.746,80.790,82.644,246.4
Drugname_SourceInfodrug_1,91.426,93.260,92.320,540.0
macro,83.382,79.600,80.444,3296.2
micro,86.400,86.400,86.400,3296.2


In [167]:
RE_gold_NER_dfs[4]

,precision,recall,f1-score,support
Drugname_Diseasename_1,83.51,95.88,89.27,729
ADR_Drugname_0,87.86,73.43,80.00,207
Diseasename_Indication_1,81.70,92.12,86.60,533
Drugname_Diseasename_0,88.10,61.67,72.55,360
ADR_Drugname_1,87.03,94.62,90.66,390
Drugname_SourceInfodrug_0,88.04,77.88,82.65,208
Drugname_SourceInfodrug_1,92.07,96.04,94.01,556
Diseasename_Indication_0,46.15,24.66,32.14,146
micro,85.17,85.17,85.17,3129
macro,81.81,77.04,78.49,3129


In [160]:
print('\n'.join(RE_gold_NER))

type                                precision       recall     f1-score      support
ADR_Drugname_1                      92.18           96.39      94.24         526
Diseasename_Indication_1            89.10           95.96      92.40         545
none_Drugname_Diseasename_0         80.87           81.58      81.22         456
Drugname_SourceInfodrug_1           93.45           91.99      92.72         512
none_ADR_Drugname_0                 88.12           76.22      81.74         185
none_Diseasename_Indication_0       55.10           30.00      38.85         90
none_Drugname_SourceInfodrug_0      83.98           86.69      85.32         248
Drugname_Diseasename_1              89.60           89.16      89.38         812
micro                               88.32           88.32      88.32         3374
macro                               84.05           81.00      81.98         3374
Parse data from /s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext/2/test.json
Parse 

In [161]:
s= '''type                                precision       recall     f1-score      support
ADR_Drugname_1                      92.18           96.39      94.24         526
Diseasename_Indication_1            89.10           95.96      92.40         545
none_Drugname_Diseasename_0         80.87           81.58      81.22         456
Drugname_SourceInfodrug_1           93.45           91.99      92.72         512
none_ADR_Drugname_0                 88.12           76.22      81.74         185
none_Diseasename_Indication_0       55.10           30.00      38.85         90
none_Drugname_SourceInfodrug_0      83.98           86.69      85.32         248
Drugname_Diseasename_1              89.60           89.16      89.38         812
micro                               88.32           88.32      88.32         3374
macro                               84.05           81.00      81.98         3374
Parse data from /s/ls4/users/romanrybka/pharm_er/true_RE/Joint/data/RDRS_multicontext/2/test.json
Parse data from /s/ls4/users/romanrybka/pharm_er/true_RE/Joint/results/RDRS_multicontext/2//RE/re_res_pred_pred_ner.json
'''
print(re.sub('Parse.+\n', '', s))

type                                precision       recall     f1-score      support
ADR_Drugname_1                      92.18           96.39      94.24         526
Diseasename_Indication_1            89.10           95.96      92.40         545
none_Drugname_Diseasename_0         80.87           81.58      81.22         456
Drugname_SourceInfodrug_1           93.45           91.99      92.72         512
none_ADR_Drugname_0                 88.12           76.22      81.74         185
none_Diseasename_Indication_0       55.10           30.00      38.85         90
none_Drugname_SourceInfodrug_0      83.98           86.69      85.32         248
Drugname_Diseasename_1              89.60           89.16      89.38         812
micro                               88.32           88.32      88.32         3374
macro                               84.05           81.00      81.98         3374



In [133]:
print(RE_gold_NER)


--- Entities (named entity recognition (NER)) ---

An entity is considered correct if the entity type and span is predicted correctly
type                                precision       recall     f1-score      support
Disease:DisTypeNegatedADE           100.00           100.00     100.00         189
Medication:MedTypeFrequency         100.00           100.00     100.00         33
Medication:MedTypeDrugclass         100.00           100.00     100.00         214
Medication:MedTypeDrugBrand         100.00           100.00     100.00         410
Disease:DisTypeBNE-Pos              100.00           100.00     100.00         380
Medication:MedTypeDrugname          100.00           100.00     100.00         734
Medication:MedTypeDosage            100.00           100.00     100.00         53
Medication:MedTypeMedMaker          100.00           100.00     100.00         106
Disease:DisTypeADE-Neg              100.00           100.00     100.00         4
ADR                                 1

In [135]:
print(RE_gold_NER.split('A relation is considered correct if the relation type and the two related entities are predicted correctly (in span and entity type)')[-1])


type                                precision       recall     f1-score      support
none_Drugname_Diseasename_0         84.54           76.28      80.20         430
none_Drugname_SourceInfodrug_0      86.57           76.95      81.48         243
Diseasename_Indication_1            81.02           96.04      87.89         480
ADR_Drugname_1                      92.33           97.22      94.71         396
none_Diseasename_Indication_0       66.67           26.03      37.44         146
Drugname_Diseasename_1              88.47           92.88      90.62         843
none_ADR_Drugname_0                 94.15           84.69      89.17         209
Drugname_SourceInfodrug_1           89.87           94.49      92.12         526
micro                               87.26           87.26      87.26         3273
macro                               85.45           80.57      81.70         3273




In [112]:
s = '''{'ADR': 56.35,
 'Disease_DisTypeADE-Neg': 0.0,
 'Disease_DisTypeBNE-Pos': 43.23,
 'Disease_DisTypeDiseasename': 85.54,
 'Disease_DisTypeIndication': 65.5,
 'Disease_DisTypeNegatedADE': 51.61,
 'Disease_DisTypeWorse': 10.0,
 'Medication_MedFromDomestic': 77.96,
 'Medication_MedFromForeign': 80.6,
 'Medication_MedMakerDomestic': 82.93,
 'Medication_MedMakerForeign': 71.19,
 'Medication_MedTypeDosage': 61.94,
 'Medication_MedTypeDrugBrand': 84.49,
 'Medication_MedTypeDrugclass': 93.42,
 'Medication_MedTypeDrugform': 92.79,
 'Medication_MedTypeDrugname': 95.49,
 'Medication_MedTypeDuration': 61.9,
 'Medication_MedTypeFrequency': 59.26,
 'Medication_MedTypeMedMaker': 89.83,
 'Medication_MedTypeRoute': 65.33,
 'Medication_MedTypeSourceInfodrug': 63.94}'''

s = re.sub("\n|\s|{|}|'", '', s)
ents_d = {ent_res.split(':')[0]:float(ent_res.split(':')[1]) for ent_res in s.split(',')}
ents_d

{'ADR': 56.35,
 'Disease_DisTypeADE-Neg': 0.0,
 'Disease_DisTypeBNE-Pos': 43.23,
 'Disease_DisTypeDiseasename': 85.54,
 'Disease_DisTypeIndication': 65.5,
 'Disease_DisTypeNegatedADE': 51.61,
 'Disease_DisTypeWorse': 10.0,
 'Medication_MedFromDomestic': 77.96,
 'Medication_MedFromForeign': 80.6,
 'Medication_MedMakerDomestic': 82.93,
 'Medication_MedMakerForeign': 71.19,
 'Medication_MedTypeDosage': 61.94,
 'Medication_MedTypeDrugBrand': 84.49,
 'Medication_MedTypeDrugclass': 93.42,
 'Medication_MedTypeDrugform': 92.79,
 'Medication_MedTypeDrugname': 95.49,
 'Medication_MedTypeDuration': 61.9,
 'Medication_MedTypeFrequency': 59.26,
 'Medication_MedTypeMedMaker': 89.83,
 'Medication_MedTypeRoute': 65.33,
 'Medication_MedTypeSourceInfodrug': 63.94}

In [121]:
df = pd.DataFrame({'f1_conll': ents_d.values()}, index=ents_d.keys())
df

,f1_conll
ADR,56.35
Disease_DisTypeADE-Neg,0.00
Disease_DisTypeBNE-Pos,43.23
Disease_DisTypeDiseasename,85.54
Disease_DisTypeIndication,65.50
Disease_DisTypeNegatedADE,51.61
Disease_DisTypeWorse,10.00
Medication_MedFromDomestic,77.96
Medication_MedFromForeign,80.60
Medication_MedMakerDomestic,82.93


In [94]:
len(re.split('====[^=]+====', txt))

5

In [92]:
inp_file

'./results_joint/RDRS_all_folds_2382077.out'

In [82]:
print(NER_test_res[2])

A


In [311]:
#читаем файл, делим на сперт репорты
with open(inp_file, 'r') as f:
    txt = f.read()

txt_parts = re.split('Evaluation\n\n|--------------------------------------------------', txt)
txt_parts = [txt_part for txt_part in txt_parts if txt_part]
spert_reports = [txt_part for txt_part in txt_parts if re.search('--- Entities .+\s+---', txt_part)]

spert_reports = [re.sub('\n\s+', '\n', rep) for rep in spert_reports]


#основной код
def count_symbols_before_first_whitespace(matchobj):
    num_symbols = matchobj.span()[0]
    symbols_before_first_whitespace.append(num_symbols)
    return '<'+str(num_symbols)+'>'

def make_valid_margin(matchobj):
    margin = max_margin - int(matchobj.group(0)[1:-1])
    return ' '*margin

def delete_n_first_whitespaces(matchobj):
    columns = matchobj.group()
    first_n_whitespaces = re.match('\s+', columns).span()[1]
    return ' '*(first_n_whitespaces-delete_n_first_whitespaces.n)+columns[first_n_whitespaces:]



for rep_id in range(len(spert_reports)):
    sents = re.split('\n+', spert_reports[rep_id])
    #sents = spert_reports[rep_id].split('\n')
    symbols_before_first_whitespace = []
    for sent_id in range(1, len(sents)):
        #считаем кол-во символов до первого пробела в каждом предложении, 
        #заменяем первую череду пробелов в предложении
        #на это кол-во символов
        if sents[sent_id].find('---')>=0 or re.search('\d\.\d', sents[sent_id]) is None and sents[sent_id].find('f1-score')<0:
            sents[sent_id] = '\n' + sents[sent_id]
            continue
        if sents[sent_id].find('MainThread')>=0:
            break
        sents[sent_id] = re.sub('\s+', count_symbols_before_first_whitespace, sents[sent_id], count=1)
        sents[sent_id] = re.sub('(?<=\d)\s+', ' '*11, sents[sent_id], count=1)
        #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
        sents[sent_id] = re.sub('(?<=\W0\.00)\s+', ' '*12, sents[sent_id], count=1)
        delete_n_first_whitespaces.n=2
        sents[sent_id] = re.sub('\s+\d+\.\d+\s+\d+$', delete_n_first_whitespaces, sents[sent_id])
        sents[sent_id] = re.sub('\s+(\d+)$', ' '*9+r'\1', sents[sent_id])
        #след. строчка для того, чтобы цифры 0.00 норм обрабатывались
        sents[sent_id] = re.sub('(?<=\W0\.00)\s+(\d+)$', ' '*10+r'\1', sents[sent_id])
        
    max_margin = max(symbols_before_first_whitespace)+4
    spert_reports[rep_id] = '\n'.join(sents)
    spert_reports[rep_id] = re.sub('<\d+>', make_valid_margin, spert_reports[rep_id])

#дампим вывод
out_file = './results/pretty_print_positive_relations.txt'


txt = '============\n'.join(spert_reports)
with open(out_file, 'w') as f:
    f.write(txt)